<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-15 15:02:20
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.21 L
Current:  1.27 C
-------------------
Today PnL: 1.08 L (0.86%)
Current PnL: -17.01 L (-12.06%)
CY Booked + Current PnL: -14.14 L (-10.02%)
-------------------
Total profit:  4.13 L
Total loss:  -21.13 L
-------------------
Total Booked + Current PnL: 12.84 L (11.26%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.27%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.06 C
Est FTT PnL: 79.47 L (62.76%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (111.35%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.94,X-MC,86.32,141060.0,26282.0,9451.0,-0.01,22.90,6.70,31.13,220.0,2.78,1.13,37.41,AR,ATH,TYRES
62,RELIANCE,1288.53,1526.00,6.87,X-LC,96.84,229827.0,26239.0,11285.0,2.12,12.89,4.91,18.43,157.0,2.33,1.85,25.19,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.88,H-LC,83.16,149898.0,24193.0,11797.0,1.67,19.25,7.87,28.63,147.0,2.05,1.20,33.54,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,37.89,93339.0,18217.0,13945.0,0.09,24.25,14.94,42.81,120.0,1.31,0.75,17.06,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-12.51,M-LC,46.32,214054.0,12574.0,14641.0,0.88,6.24,6.84,13.51,60.0,0.86,1.72,17.27,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-837.29,H-SC,100.00,70249.0,-25067.0,69764.0,6.65,-26.30,99.31,46.89,82.0,-0.36,0.56,38.56,X40N,NTT,MISC
89,VALIANTORG,512.64,838.00,-308.98,H-SC,8.42,88920.0,-44366.0,128961.0,4.64,-33.29,145.03,63.47,161.0,-0.34,0.71,49.05,XR,NTT,CHEMICALS
34,HONAUT,42646.15,58518.52,-22.88,X-MC,72.63,112980.0,-14958.0,62580.0,4.38,-11.69,55.39,37.22,73.0,-0.24,0.91,15.35,X40N,ATH,ELECTRICAL
76,STARHEALTH,551.56,761.00,-7.05,M-SC,67.37,152170.0,-48046.0,124079.0,3.87,-24.00,81.54,37.97,62.0,-0.39,1.22,22.18,XY24,NTT,INSURANCE
81,TATAMOTORS,776.02,1065.00,-52.03,M-LC,94.74,192178.0,-13467.0,90055.0,3.76,-6.55,46.86,37.24,1.0,-0.15,1.54,25.09,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,SURYODAY,189.78,240.00,40.35,H-SC,51.58,100054.0,-49872.0,89548.0,-2.95,-33.26,89.50,26.46,88.0,-0.56,0.80,28.18,XR,NTT,BANKS
58,QUESS,711.66,986.00,NaN,NaN,97.89,78358.0,-77496.0,137573.0,-2.84,-49.72,175.57,38.55,NaN,-0.56,0.63,19.99,XY24,NTT,MISC
79,TANLA,917.30,1963.11,-40.61,H-SC,95.79,145534.0,-93881.0,366833.0,-1.79,-39.21,252.06,114.01,20.0,-0.26,1.17,33.48,AR,ATH,IT
71,SFL,1016.35,1287.00,272.62,H-SC,20.00,168216.0,-94002.0,163826.0,-1.75,-35.85,97.39,26.63,58.0,-0.57,1.35,1.23,XY24,NTT,MISC
87,UNIONBANK,123.87,163.00,-14.11,M-LC,61.05,150880.0,10040.0,34446.0,-1.65,7.13,22.83,31.59,48.0,0.29,1.21,31.06,XY24,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-52.03,M-LC,94.74,192178.0,-13467.0,90055.0,3.76,-6.55,46.86,37.24,1.0,-0.15,1.54,25.09,XY24,NTT,AUTO
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,66.32,102670.0,943.0,113430.0,2.17,0.93,110.48,112.43,2.0,0.01,0.83,21.17,AR,ATH,MISC
2,ABB,5319.37,8953.12,-37.34,H-LC,70.53,143262.0,10278.0,80571.0,1.69,7.73,56.24,68.31,4.0,0.13,1.15,16.82,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-14.23,X-LC,71.58,193201.0,-18927.0,76681.0,0.82,-8.92,39.69,27.23,5.0,-0.25,1.55,10.72,X40,BTT,IT
48,LICI,777.74,983.00,-15.35,H-LC,55.79,187936.0,13722.0,32250.0,2.04,7.88,17.16,26.39,6.0,0.43,1.51,15.06,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.94,X-MC,86.32,141060.0,26282.0,9451.0,-0.01,22.90,6.70,31.13,220.0,2.78,1.13,37.41,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,7.09,X-MC,12.63,188678.0,33243.0,37188.0,-0.13,21.39,19.71,45.32,201.0,0.89,1.52,28.38,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,37.89,93339.0,18217.0,13945.0,0.09,24.25,14.94,42.81,120.0,1.31,0.75,17.06,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,8.17,H-MC,88.42,169232.0,29066.0,41919.0,1.05,20.74,24.77,50.65,55.0,0.69,1.36,29.16,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,91.58,187937.0,36731.0,28942.0,0.19,24.29,15.40,43.43,37.0,1.27,1.51,30.27,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.16,M-SC,26.32,138111.0,533.0,93874.0,-0.14,0.39,67.97,68.62,79.0,0.01,1.11,19.73,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,8.17,H-MC,88.42,169232.0,29066.0,41919.0,1.05,20.74,24.77,50.65,55.0,0.69,1.36,29.16,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,2.48,H-SC,73.68,166855.0,15491.0,30284.0,0.72,10.23,18.15,30.25,111.0,0.51,1.34,32.06,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,91.58,187937.0,36731.0,28942.0,0.19,24.29,15.40,43.43,37.0,1.27,1.51,30.27,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-4.76,H-SC,35.79,140378.0,2518.0,35249.0,0.54,1.83,25.11,27.39,68.0,0.07,1.13,24.54,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WIPRO,243.46,420.00,-6.99,M-LC,43.16,159036.0,8091.0,101370.0,1.41,5.36,63.74,72.51,52.0,0.08,1.28,19.74,XR,NTT,IT
18,CAMS,3643.00,5250.99,-7.33,H-SC,11.58,104194.0,2190.0,42834.0,1.98,2.15,41.11,44.14,22.0,0.05,0.84,18.74,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-52.59,H-SC,66.32,102670.0,943.0,113430.0,2.17,0.93,110.48,112.43,2.0,0.01,0.83,21.17,AR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,31.58,129595.0,443.0,86621.0,0.02,0.34,66.84,67.42,41.0,0.01,1.04,26.04,XR,NTT,BANKS
92,VOLTAS,1272.00,1929.20,-30.46,H-MC,14.74,138930.0,-990.0,73286.0,1.16,-0.71,52.75,51.67,32.0,-0.01,1.12,4.94,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,41.19,M-SC,85.26,122931.0,-19548.0,39067.0,1.40,-13.72,31.78,13.70,214.0,-0.50,0.99,45.09,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,14.94,X-MC,86.32,141060.0,26282.0,9451.0,-0.01,22.90,6.70,31.13,220.0,2.78,1.13,37.41,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,0.88,H-LC,83.16,149898.0,24193.0,11797.0,1.67,19.25,7.87,28.63,147.0,2.05,1.20,33.54,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-14.11,M-LC,61.05,150880.0,10040.0,34446.0,-1.65,7.13,22.83,31.59,48.0,0.29,1.21,31.06,XY24,NTT,BANKS
14,BERGEPAINT,558.54,680.00,-15.93,H-MC,60.00,180532.0,3475.0,35023.0,2.99,1.96,19.40,21.75,154.0,0.10,1.45,29.14,XY24,NTT,PAINTS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-837.29,H-SC,100.00,70249.0,-25067.0,69764.0,6.65,-26.30,99.31,46.89,82.0,-0.36,0.56,38.56,X40N,NTT,MISC
80,TATAELXSI,7332.28,9000.00,-17.49,X-MC,98.95,86604.0,-16048.0,39396.0,1.40,-15.63,45.49,22.74,47.0,-0.41,0.70,30.18,X40N,NTT,IT
58,QUESS,711.66,986.00,NaN,NaN,97.89,78358.0,-77496.0,137573.0,-2.84,-49.72,175.57,38.55,NaN,-0.56,0.63,19.99,XY24,NTT,MISC
62,RELIANCE,1288.53,1526.00,6.87,X-LC,96.84,229827.0,26239.0,11285.0,2.12,12.89,4.91,18.43,157.0,2.33,1.85,25.19,XY25,NTT,REFINERIES
79,TANLA,917.30,1963.11,-40.61,H-SC,95.79,145534.0,-93881.0,366833.0,-1.79,-39.21,252.06,114.01,20.0,-0.26,1.17,33.48,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.52
1,25,39.22
2,50,68.31


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.98
X40N    15.31
X40     14.49
AR      14.10
XR      10.35
XY25     8.42
X200     1.21
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.46
M-SC    17.85
X-LC    16.42
M-LC     8.43
H-MC     7.87
X-MC     6.79
H-LC     5.96
M-MC     5.58
X-SC     2.82
L-SC     2.28
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.73,-1.66,33.99
BANKS,11.11,-8.94,62.37
IT,9.32,-19.36,85.95
FINANCE,7.48,-19.08,72.81
MISC,7.17,-27.06,81.73
INSURANCE,6.15,-0.65,30.99
PAINTS,5.65,-16.80,51.37
ELECTRICAL,4.16,-7.69,62.57
HEALTHCARE,3.64,-2.69,35.85


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2997789.0,27
XR,1256918.0,15
X40N,1256220.0,18
AR,1251052.0,15
X40,788994.0,12
SR,206611.0,2
XY25,121813.0,5
X200,67417.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2391962.0,22
M-SC,2015866.0,21
X-LC,699977.0,12
M-MC,573295.0,5
H-MC,416186.0,7
X-SC,357618.0,4
M-LC,330682.0,6
L-SC,314671.0,4
X-MC,310343.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      913069.0      6
M-SC       XY24      758680.0      6
H-SC       X40N      544744.0      7
           AR        541245.0      3
M-SC       X40N      449811.0      6
           XR        428532.0      5
M-MC       XY24      403857.0      3
X-LC       X40       369615.0      5
H-SC       XR        300345.0      4
M-LC       XY24      214671.0      4
M-SC       X40       199274.0      2
H-LC       AR        197879.0      4
X-LC       AR        188023.0      3
H-MC       XY24      173298.0      2
L-SC       XR        162914.0      2
X-MC       XY24      140967.0      2
X-SC       SR        120798.0      1
           XR        119050.0      1
           X40N      117770.0      2
X-MC       X40N      101976.0      2
M-LC       XR        101370.0      1
M-SC       AR         93756.0      1
H-SC       X40        92559.0      2
L-SC       XY24       87162.0      1
M-MC       XR         86621.0      1
M-SC       SR         85813.0      1
M-MC       AR         82817.0      1
X-LC       XY25       74922.0      3
H-MC       AR         73286.0      1
           X40        69597.0      2
X-LC       X200       67417.0      1
L-SC       AR         64595.0      1
H-MC       XR         58086.0      1
X-MC       X40        57949.0      1
H-MC       X40N       41919.0      1
H-LC       XY25       32250.0      1
M-LC       XY25       14641.0      1
X-MC       AR          9451.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
